Import Data

In [65]:
import wpparser
#down_path = "C:\\Users\\kales\\Downloads\\"
#test_data = wpparser.parse(down_path + "tear2023.wordpress.com-2023-12-15-19_01_51-1qoarrtidqhyn2xegrnqyfjiface6xv2\\tear2023.wordpress.com-2023-12-15-19_01_35\\text.xml")
data = wpparser.parse(".\\tear2023.wordpress.com-2023-12-15-19_01_51-1qoarrtidqhyn2xegrnqyfjiface6xv2\\tear2023.wordpress.com-2023-12-15-19_01_35\\tear2023.wordpress.2023-12-15.000.xml")

Create Image Data Dictionary 

In [66]:
import re
img_pat = r'files.wordpress.com/(.*?)"'
img_path_re = re.compile(img_pat)
all_img_data = []
for post in data['posts']:
    if post['content'] is None:
        cont = img_path_re.findall(post['guid'] + '\"', re.DOTALL)
        img_data = {
            'id': post['post_id'],
            'width': post['postmeta']['attachment_metadata'][b'width'],
            'height': post['postmeta']['attachment_metadata'][b'height'],
            'path': cont[0]
        }
        all_img_data.append(img_data)

Create Post Content Dictionary

In [67]:
import re

paragraphs = r'<p>(.*?)</p>'
img = r'<img\s+src="https://tear2023.files.wordpress.com/(.*?)\?w='
img_re = re.compile(img)
#regex =  r'<p>(.*?)</p>|<img\s+src="https://tear2023.files.wordpress.com/(.*?)\?w='
regex =  r'<p>(.*?)</p>|class="wp-image-(.*?)" /'
all_re = re.compile(regex)
post_content = ""
posts_content = []
para_re = re.compile(paragraphs)
for post in data['posts']:
    if (post['content'] is not None):
        fImage_id = ""
        try:
            fImage_id = post['postmeta']['featured_img_id']
        except KeyError:
            fImage_id = ""
        post_content = {
            'title': post['title'],
            'content': all_re.findall(post['content'], re.DOTALL),
            'post_id': post['post_id'],
            'featured_img': fImage_id,
        }
        posts_content.append(post_content)
        #if (post['title'] == "Day 100/101: Finale"):
            #print(post['content'])
            #print(post_content['content'][1][1])
#print(data['posts'][-5])
print(posts_content[5]['content'][4][1])

108


In [68]:
up_posts_content = []
for post in posts_content:
    new_post = post.copy()
    obj_num = 0
    new_post['content'] = []
    for obj in post['content']:
        if not (obj[1]) == "":
            img_cont = {}
            for img in all_img_data:
                if img['id'] == obj[1]:
                    #print('check')
                    img_cont = img.copy()
                    break
            new_post['content'].append(("",img_cont))
        else:
            new_post['content'].append(obj)
        obj_num += 1
    up_posts_content.append(new_post)

In [69]:
print(up_posts_content[5])

{'title': 'Seattle to London ✈️', 'content': [('Yesterday morning Kale Sophie and I left for London.', ''), ('', {'id': '89', 'width': 4032, 'height': 3024, 'path': '2023/06/img_0310.jpg'}), ('', {'id': '90', 'width': 3024, 'height': 4032, 'path': '2023/06/img_0309.jpg'}), ('', {'id': '91', 'width': 4032, 'height': 3024, 'path': '2023/06/img_5597.jpg'}), ('', {'id': '108', 'width': 3024, 'height': 3024, 'path': '2023/06/image_editor_output_image1650214385-1686831258266.jpg'}), ('We first had a brief layover in Calgary before our 8 hour flight to London. The flight was long, but between a few naps and watching the new Antman it went by fairly quick. We got through the Gatwick airport around 10am, and took a train into the city. I had an appointment to get my encephalitis shot at 2:30, so we spent our time beforehand walking from the London Bridge to the clinic, stopping at a breakfast place and a park along the way. Kale and I both got English breakfasts, which did not disappoint. ', ''

Print Data to Docx

In [103]:

from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from PIL import Image
from io import BytesIO
#img_path = "C:/Users/kales/Downloads/media-export-220074516-from-0-to-2745/"
img_path = "./media-export-220074516-from-0-to-2745/"
doc = Document()
doc.add_heading('Tear 2023', 0)
doc.add_heading('By: Hunter Hatzenbeler, Kale Strange, Sam Shepard, and Sophie Harris', level=1)
imagesList = []
img_num = 0
for post in up_posts_content[0:8]:
    #print(post)
    if not ((post['title'] == 'Dinaric Alps Gallery') or (post['title'] == 'Swiss Alps Gallery')):
        doc.add_section()
        doc.add_heading(post['title'], level=1)
        #if len(post['content']) > 1:
        #    if post['content'][1] == "":
        #        builder.writeln(post['content'][0])
        print(post['title'])
        fImagePath = ""
        if not (post['featured_img'] == ""):
            imgID = post['featured_img']
            for img in all_img_data:
                if (img['id'] == imgID):
                    fImagePath = img['path']
                    #Decrease image resolution by factor of 5 (for testing). It takes longer, but much smaller file size 
                    temp_image = Image.open(img_path + fImagePath)
                    im_w, im_h = temp_image.size
                    im_resized = temp_image.resize((int(im_w/5), int(im_h/5)), Image.Resampling.LANCZOS)
                    #print(im_resized.tobytes("raw"))
                    #img_bytes = BytesIO(im_resized.tobytes("raw"))
                    img_bytes = BytesIO()
                    im_resized.save(img_bytes, format="PNG")
                    #print(img_bytes)
                    doc.add_picture(img_bytes, width=Inches(4))
                    #doc.add_picture((img_path + fImagePath), width=Inches(4))
                    doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
                    break
        for content in post['content']:
            #img_num = 1
            if content[1] == "":
                if not img_num == 0:
                    print_images(imagesList, doc)
                imagesList = []
                doc.add_paragraph(content[0])
                img_num = 0
            else: 
                #print(content[1])
                if not (content[1]['id'] == post['featured_img']):
                    imagesList.append(content[1])
                    img_num += 1
                    """
                    ext_img_path = content[1]['path']
                    temp_image = Image.open(img_path + ext_img_path)
                    im_w, im_h = temp_image.size
                    im_resized = temp_image.resize((int(im_w/5), int(im_h/5)), Image.Resampling.LANCZOS)
                    img_bytes = BytesIO()
                    im_resized.save(img_bytes, format="PNG")
                    if (img_num == 0):
                        doc.add_picture(img_bytes, width=Inches(2.5))
                        #doc.add_picture((img_path + content[1]), width=Inches(2.5))
                        doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
                        img_num = 1
                    else:
                        newRun = doc.paragraphs[-1].add_run()
                        newRun.add_picture(img_bytes, width=Inches(2.5))
                        img_num = 1
                    """
        if not img_num == 0:
            print_images(imagesList, doc)
        imagesList = []
        img_num = 0
#doc.update_fields()
doc.save("out.docx")

About
T-minus 24 Hours
The final supper
Seattle to London ✈️
Shep out
Day 1: Reunions and Bulgaria


In [102]:
def print_images(images, doc):
    img_num = 0
    total_width = 5
    image_makeup = {
        'tall': [],
        'short': []
    }
    for image in images:
        if float(image['width']) >= float(img['height']):
            image_makeup['short'].append(image)
        else:
            image_makeup['tall'].append(image)
    num_short = len(image_makeup['short'])
    num_tall = len(image_makeup['tall'])
    for image in image_makeup['short']:
        if (img_num == 0):
            prevImg = [img_to_bytes(image), float(image['width']), float(image['height'])]
            img_num = 1
        else:
            newImg = img_to_bytes(image)
            #make image height equal
            hRatio = int(image['height'])/prevImg[2]
            wratio = int(image['width'])/(int(image['width'])+(prevImg[1] * hRatio))
            doc.add_picture(prevImg[0], width=Inches(total_width*(1-wratio)))
            doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
            newRun = doc.paragraphs[-1].add_run()
            newRun.add_picture(newImg, width=Inches(total_width*wratio))
            img_num = 0
    if img_num == 1:
        doc.add_picture(prevImg[0], width=Inches(2.5))
        doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
    for image in image_makeup['tall']:
        imBytes = img_to_bytes(image)
        doc.add_picture(imBytes, width=Inches(2.5))
        doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER

    """
    for image in images:
        ext_img_path = image['path']
        temp_image = Image.open(img_path + ext_img_path)
        im_w, im_h = temp_image.size
        im_resized = temp_image.resize((int(im_w/10), int(im_h/10)), Image.Resampling.LANCZOS)
        img_bytes = BytesIO()
        im_resized.save(img_bytes, format="PNG")
        if (img_num == 0):
            doc.add_picture(img_bytes, width=Inches(2.5))
            #doc.add_picture((img_path + ext_img_path), width=Inches(2.5))
            doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
            img_num = 1
        else:
            newRun = doc.paragraphs[-1].add_run()
            newRun.add_picture(img_bytes, width=Inches(2.5))
            img_num = 0
    """

In [96]:
def img_to_bytes(image):
    ext_img_path = image['path']
    temp_image = Image.open(img_path + ext_img_path)
    im_w, im_h = temp_image.size
    im_resized = temp_image.resize((int(im_w/10), int(im_h/10)), Image.Resampling.LANCZOS)
    img_bytes = BytesIO()
    im_resized.save(img_bytes, format="PNG")
    return img_bytes